In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import openai
import pandas as pd
from openai import OpenAI

# Initialize OpenAI client (uses OPENAI_API_KEY from .env)
client = OpenAI()

In [3]:
# Sample dataset (you can load from CSV, JSON, or database)
data = pd.read_csv("LimitedData.csv")  
df = pd.DataFrame(data)
print("Dataset:")
print(df)

/var/folders/jl/s_hrs6hd5p76qccvcrb_v_400000gn/T/ipykernel_45985/1863192452.py:2: DtypeWarning: Columns (0: DESYNPUF_ID, 1: BENE_ESRD_IND, 2: ICD9_DGNS_CD_1, 3: LINE_PRCSG_IND_CD_1, 4: LINE_ICD9_DGNS_CD_1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("LimitedData.csv")


Dataset:
             DESYNPUF_ID  BENE_BIRTH_DT  BENE_DEATH_DT  BENE_SEX_IDENT_CD  \
0       BAC397A752FA03A7     19330101.0            0.0                2.0   
1       BE4841C03C968C6B     19341201.0            0.0                2.0   
2       0DCFC21B16C7A18F     19300901.0            0.0                2.0   
3       5ED5E5248B290111     19220201.0            0.0                2.0   
4       7FA31DE58F7B26E1     19280801.0            0.0                2.0   
...                  ...            ...            ...                ...   
999895               NaN            NaN            NaN                NaN   
999896               NaN            NaN            NaN                NaN   
999897               NaN            NaN            NaN                NaN   
999898               NaN            NaN            NaN                NaN   
999899               NaN            NaN            NaN                NaN   

        BENE_RACE_CD BENE_ESRD_IND  SP_STATE_CODE  BENE_COUNTY_CD 

In [ ]:
# Call OpenAI API to get insights from the dataset
def get_dataset_insights(dataframe):
    """
    Send dataset to OpenAI and get insights back.
    
    Args:
        dataframe: pandas DataFrame with the data
    
    Returns:
        str: Insights from OpenAI
    """
    # Convert DataFrame to CSV string for sending to API
    prompt = f"""

    """

    
    # Create the prompt
    prompt = f"""Analyze the following dataset and provide key insights, trends, and recommendations:

Dataset:
    Analyze this dataset structure and summary:

    Columns:
    {df.columns.tolist()}

    Data types:
    {df.dtypes}

    Summary statistics:
    {df.describe().to_string()}

    Sample rows:
    {df.head(20).to_string()}

Please provide:
1. Summary statistics
2. Key trends and patterns
3. Recommendations for improvement
4. Any anomalies or areas of concern"""
    
    # Call OpenAI API using chat completions
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # or use "gpt-4" for more advanced analysis
        messages=[
            {"role": "system", "content": "You are a data analyst assistant. Provide clear, actionable insights."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=1000
    )
    
    # Extract and return the insights
    insights = response.choices[0].message.content
    return insights

# Get insights from your dataset
print("Requesting insights from OpenAI...\n")
insights = get_dataset_insights(df)
print("Insights:")
print(insights)

Requesting insights from OpenAI...



### Making a Powerpoint of Insights

In [11]:
insights.split("###")

['',
 " 1. Summary Statistics\nThe provided dataset contains 1,000 records with various attributes related to beneficiaries, coverage, and claims. Here are key summary statistics based on the columns:\n\n- **Beneficiary Age**: The average birth date appears to be around 1933, with some beneficiaries born as early as 1922 and as late as 1961. The mean age of the beneficiaries is approximately 90 years.\n- **Death Dates**: The majority of entries have no recorded death date (0.0), indicating that those beneficiaries are alive or the data is incomplete.\n- **Gender Distribution**: The `BENE_SEX_IDENT_CD` has a mean value of 1.57, suggesting a near-equal distribution between male (1) and female (2) beneficiaries.\n- **Race Distribution**: The `BENE_RACE_CD` indicates diversity, with values ranging from 1 to 5, corresponding to different racial categories.\n- **Chronic Conditions**: The dataset includes various health conditions (e.g., Alzheimer's, CHF, COPD, Diabetes) coded as binary indic

In [12]:
!pip install python-pptx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 27.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [python-pptx] [python-pptx]


In [13]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor

def create_powerpoint_from_insights(insights_text, output_file="dataset_insights.pptx"):
    """
    Create a PowerPoint presentation from OpenAI insights.
    
    Args:
        insights_text: str - The insights text from OpenAI
        output_file: str - Output filename for the presentation
    
    Returns:
        str - Path to the created PowerPoint file
    """
    # Create a presentation
    prs = Presentation()
    prs.slide_width = Inches(10)
    prs.slide_height = Inches(7.5)
    
    # Title Slide
    title_slide_layout = prs.slide_layouts[0]  # Title slide layout
    slide = prs.slides.add_slide(title_slide_layout)
    title = slide.shapes.title
    subtitle = slide.placeholders[1]
    
    title.text = "Dataset Analysis Insights"
    subtitle.text = "Generated by OpenAI Data Analysis"
    
    # Parse insights into sections (split by numbers like "1.", "2.", etc.)
    sections = insights_text.split('\n')
    current_section = []
    
    # Content Slides
    for line in sections:
        if line.strip():  # Non-empty line
            current_section.append(line.strip())
        
        # Create a new slide when we hit a numbered section or have accumulated enough text
        if (line.strip().startswith(('1.', '2.', '3.', '4.', '5.', '###')) 
            and current_section and len(current_section) > 1):
            
            # Add slide with content
            blank_slide_layout = prs.slide_layouts[5]  # Blank layout
            slide = prs.slides.add_slide(blank_slide_layout)
            
            # Add title (first line as section title)
            title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(9), Inches(1))
            title_frame = title_box.text_frame
            title_frame.word_wrap = True
            p = title_frame.paragraphs[0]
            p.text = current_section[0]
            p.font.size = Pt(32)
            p.font.bold = True
            p.font.color.rgb = RGBColor(0, 51, 102)  # Dark blue
            
            # Add content
            content_box = slide.shapes.add_textbox(Inches(0.5), Inches(1.8), Inches(9), Inches(5.2))
            content_frame = content_box.text_frame
            content_frame.word_wrap = True
            
            for content_line in current_section[1:]:
                p = content_frame.add_paragraph()
                p.text = content_line
                p.level = 0
                p.font.size = Pt(18)
                p.space_before = Pt(6)
                p.space_after = Pt(6)
            
            current_section = [line.strip()] if line.strip().startswith(('1.', '2.', '3.', '4.', '5.', '###')) else []
    
    # Add remaining content as final slide
    if current_section:
        blank_slide_layout = prs.slide_layouts[5]
        slide = prs.slides.add_slide(blank_slide_layout)
        
        title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(9), Inches(1))
        title_frame = title_box.text_frame
        p = title_frame.paragraphs[0]
        p.text = current_section[0]
        p.font.size = Pt(32)
        p.font.bold = True
        p.font.color.rgb = RGBColor(0, 51, 102)
        
        content_box = slide.shapes.add_textbox(Inches(0.5), Inches(1.8), Inches(9), Inches(5.2))
        content_frame = content_box.text_frame
        content_frame.word_wrap = True
        
        for content_line in current_section[1:]:
            p = content_frame.add_paragraph()
            p.text = content_line
            p.font.size = Pt(18)
            p.space_before = Pt(6)
            p.space_after = Pt(6)
    
    # Save the presentation
    prs.save(output_file)
    print(f"PowerPoint presentation created: {output_file}")
    return output_file

# Generate the PowerPoint from insights
pptx_file = create_powerpoint_from_insights(insights)
print(f"✅ Presentation ready: {pptx_file}")

PowerPoint presentation created: dataset_insights.pptx
✅ Presentation ready: dataset_insights.pptx
